In [1]:
from fastai import *
from fastai.column_data import *
from fastai import conv_learner
import numpy as np
import pandas as pd
from fastai.io import *
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
PATH='nietzsche.txt'

In [3]:
txt=open(PATH).read()
print (len(txt))

600893


In [4]:
chars=sorted(list(set(txt)))
chars.insert(0,"\0")
print (f"Total chars are {len(chars)}")
vocab_size=len(chars)

Total chars are 85


In [5]:
chars_id={ch:i for i,ch in enumerate(chars)}
id_chars={i:ch for i,ch in enumerate(chars)}

In [6]:
idx=[chars_id[ix] for ix in txt ]

In [7]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
cs=3
x1=[idx[ix] for ix in range(0,len(txt)-cs,cs)]
x2=[idx[ix+1] for ix in range(0,len(txt)-cs,cs)]
x3=[idx[ix+2] for ix in range(0,len(txt)-cs,cs)]
x4=[idx[ix+3] for ix in range(0,len(txt)-cs,cs)]

In [10]:
x1[:10]

[40, 30, 29, 1, 40, 43, 31, 61, 2, 74]

In [11]:
inp1=np.array(x1[:-2])
inp2=np.array(x2[:-2])
inp3=np.array(x3[:-2])

out=np.array(x4[:-2])

In [12]:
inp1.shape

(200295,)

In [8]:
n_hidden=256
n_fac =42

In [26]:
class char3model(nn.Module):
    def __init__(self,n_hidden,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.l_in= nn.Linear(n_fac,n_hidden)
        self.l_hidden=nn.Linear(n_hidden,n_hidden)
        self.out=nn.Linear(n_hidden,vocab_size)
        
    def forward(self,c1,c2,c3):
        inp1=F.relu(self.l_in(self.e(c1)))
        inp2=F.relu(self.l_in(self.e(c2)))
        inp3=F.relu(self.l_in(self.e(c3)))
        h=V(torch.zeros(inp1.size()))
        h=F.tanh(self.l_hidden(inp1+h))
        h=F.tanh(self.l_hidden(h+inp2))
        h=F.tanh(self.l_hidden(h+inp3))
        out=self.out(h)
        return F.log_softmax(out)

In [9]:
md=ColumnarModelData.from_arrays('.',[-1],np.stack([inp1,inp2,inp3],axis=1),out,bs=32) 

NameError: name 'inp1' is not defined

In [33]:
m= char3model(n_hidden,n_fac)

In [34]:
opt=torch.optim.Adam(m.parameters(),1e-2)

In [35]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      3.260534   6.292563  



[array([6.29256])]

In [31]:
set_lrs(opt,1e-3)
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      2.537892   6.492347  



[array([6.49235])]

In [42]:
it=iter(md.trn_dl)
*xs,ys=next(it)

In [45]:
len(xs)

3

In [113]:
def test(ts):
    assert(len(ts)==3),"3 characters req"
    ex=[V(T(chars_id[ch])) for ch in ts]
    idx=to_np(m(ex[0],ex[1],ex[2])).argmax()
    return id_chars[idx]

In [118]:
test("mor")

'a'

In [158]:
cs=10
xs=np.array([idx[ix:cs+ix] for ix in range(0,len(idx)-cs)])

In [159]:
x=xs[:,:9]
y=xs[:,9]

In [160]:
y.shape

(600883,)

In [163]:
md.trn_ds.xs

[array([40, 42, 29, ..., 68, 59,  2]),
 array([42, 29, 30, ..., 59,  2, 72]),
 array([29, 30, 25, ...,  2, 72, 62]),
 array([30, 25, 27, ..., 72, 62, 67]),
 array([25, 27, 29, ..., 62, 67, 59]),
 array([27, 29,  1, ..., 67, 59, 74]),
 array([29,  1,  1, ..., 59, 74, 65]),
 array([ 1,  1,  1, ..., 74, 65, 67]),
 array([ 1,  1, 43, ..., 65, 67, 58])]

In [196]:
x[0]

array([40, 42, 29, 30, 25, 27, 29,  1,  1])

In [165]:
d=iter(md.trn_dl)
*k,l=next(d)

In [175]:
*k,l=next(d)
for n in range(len(k)):
    print (id_chars[k[n][0]],end='')
print (id_chars[l[0]],end='')

erness and

In [162]:
md=ColumnarModelData.from_arrays('.',[-1],x,y,bs=512)

In [13]:
class char_loop(nn.Module):
    def __init__(self,vocab_size,n_hidden,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.l_in=nn.Linear(n_fac,n_hidden)
        self.l_hidden=nn.Linear(n_hidden,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
    
    def forward(self,*cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden))
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [15]:
model=char_loop(vocab_size,n_hidden,n_fac)
opt=torch.optim.Adam(model.parameters())
fit(model,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      1.780601   0.292267  


[array([0.29227])]

In [116]:
set_lrs(opt,1e-3)
fit(model,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      1.646296   0.129284  



[array([0.12928])]

In [119]:
def get_next(inp):
    ex=[ V(T(chars_id[x])) for x in inp]
    #char_id=np.argmax(to_np(model(ex[0],ex[1],ex[2],ex[3],ex[4],ex[5],ex[6])))
    char_id=np.argmax(to_np(model(*ex)))
    return id_chars[char_id]

In [123]:
get_next("youdf")

'e'

In [23]:
class char_concat_loop(nn.Module):
    def __init__(self,vocab_size,n_hidden,n_fac):
        super().__init__()
        self.n_fac=n_fac
        self.n_hidden=n_hidden
        self.e=nn.Embedding(vocab_size,n_fac)
        self.l_in=nn.Linear(n_hidden+n_fac,n_hidden)
        self.l_hidden=nn.Linear(n_hidden,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
    
    def forward(self,*cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, self.n_hidden))
        for c in cs:
            inp = torch.cat((h,self.e(c)),1)
            inp= F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1 )

In [35]:
model=char_concat_loop(vocab_size,n_hidden,n_fac)

In [36]:
opt=torch.optim.Adam(model.parameters(),1e-3)
fit(model,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      1.767494   0.350444  


[array([0.35044])]

In [124]:
get_next("Th")

'i'

In [107]:
class CharRnn1(nn.Module):
    def __init__(self,vocab_size,n_hidden,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.rnn=nn.RNN(n_fac,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
    
    def forward(self,*cs):
        bs=cs[0].size(0)
        h=V(torch.zeros(1,bs,n_hidden))
        inp=self.e(torch.stack(cs))
        outp,h=self.rnn(inp,h)
        
        return F.log_softmax(self.l_out(outp[-1]))

In [108]:
m=CharRnn1(vocab_size,n_hidden,n_fac)
opt=optim.Adam(m.parameters(),1e-3)

In [109]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      1.798847   0.311293  


[array([0.31129])]

In [211]:
def get_next_n(inp,n):
    txt=inp
    for ix in range(n):
        ex=[ V(T(chars_id[x])) for x in txt[ix:]]
        char_id=np.argmax(to_np(model(*ex)))
        txt = txt+id_chars[char_id]
    return txt

In [234]:
get_next_n("therre",200)

'therreation a sight a stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stand and stan'

In [177]:
class CharRnn(nn.Module):
    def __init__(self,vocab_size,n_hidden,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.rnn=nn.RNN(n_fac,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
    
    def forward(self,*cs):
        bs=cs[0].size(0)
        h=V(torch.zeros(1,bs,n_hidden))
        inp=self.e(torch.stack(cs))
        out,h=self.rnn(inp,h)
        return F.log_softmax(self.l_out(out[-1]))

In [178]:
model=CharRnn(vocab_size,n_hidden,n_fac)
otp=torch.optim.Adam(model.parameters(),1e-2)
fit(model,md,1,otp,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      2.038243   0.964115  



[array([0.96412])]

In [103]:
set_lrs(opt,1e-3)
fit(model,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                                
    0      1.76661    0.249625  


[array([0.24963])]

In [181]:
get_next_n("the wor",200)

'the worg and in the preances of the preances of the preances of the preances of the preances of the preances of the preances of the preances of the preances of the preances of the preances of the preances of'

# Multi output

In [198]:
cs=10
xs=np.array([idx[i:cs+i] for i in range(0,len(idx)-cs-1,cs)])
ys=np.array([idx[i+1:1+cs+i] for i in range(0,len(idx)-cs,cs)])

In [200]:
md=ColumnarModelData.from_arrays('.',get_cv_idxs(len(xs)),xs,ys,bs=512)

In [263]:
class CharRnn_multi(nn.Module):
    def __init__(self,vocab_size,n_hidden,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.rnn=nn.RNN(n_fac,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
    
    def forward(self,*cs):
        bs=cs[0].size(0)
        h=V(torch.zeros(1,bs,n_hidden))
        inp=self.e(torch.stack(cs))
        out,h=self.rnn(inp,h)
        return F.log_softmax(self.l_out(out))

In [281]:
m=CharRnn_multi(vocab_size,n_hidden,n_fac)
opt=torch.optim.Adam(m.parameters(),1e-3)

# init identity

In [282]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.FloatTensor of size 256x256]

In [283]:
def nll_seq(inp,tar):  #inp: (seq,bs,vocab_size) out : (bs,seq_len)
    seq,bs,nh = inp.size()
    tar= tar.transpose(0,1).contiguous().view(-1)
    inp=inp.view(-1,nh)
    return F.nll_loss(inp,tar)

In [287]:
fit(m,md,2,opt,nll_seq)

epoch      trn_loss   val_loss                             
    0      0.918315   0.901285  
    1      0.861438   0.845668                             



[array([0.84567])]

In [285]:
def get_words(rn,n,inp):
    txt=inp
    for ix in range(0,n,cs):
        inp=[chars_id[u] for u in txt[ix:ix+cs]]
        if (len(inp)==0):
            break
        o=to_np(m(*V(T(inp)))).reshape(len(inp),85).argmax(axis=1)
        for iy in o:
            txt = txt + id_chars[iy]
    return txt

In [286]:
print (get_words(m,100,"omg i is aw"))

omg i is awm.-i6Js
alm.AMshe
obviAsOhexarAizch=ExfriMzchaEdfrizc"hatd?rizchSat!, izchaTty, (zchakly,
(4Nhakly, 
